In [23]:
import pandas as pd

**Cohort means all players who installed the game on the same day. Cohort numeration in this task starts with 3001 (for players who installed on January 1st 2017)**<br>

<font color='green'>**Когорта ( в данном случае ) - группа, людей, установивших игру в один и тот же день. Нумерация в задаче начинается с 3001 (для игроков, которые установили игру 1 января 2017 года).**</font>

In [24]:
installs = pd.read_csv('installs.csv')
ecpm = pd.read_csv('eCPM.csv')
iap = pd.read_csv('in_game_purchases_revenue.csv')
total_ads = pd.read_csv('total_ads_watched.csv')

cohorts = installs['cohort_number']

days = pd.unique(pd.concat([iap['Date'],total_ads['Date'],ecpm['Date']]))

In [25]:
# установки

installs.head(2)

,Date,cohort_number,installs
0,1/1/2017,3001,62078
1,1/2/2017,3002,62601


**CPM - Cost Per Mille<br>**
<font color='green'>**Цена за тысячу показов баннера, объявления или любого другого коммерческого блока.**</font>

In [26]:
ecpm.head(2)

,Date,eCPM
0,1/1/2017,5.879974
1,1/2/2017,5.259028


**IAP - in-app purchases**<br>
<font color='green'>**Покупки в приложении**</font>

In [27]:
iap.head(2)

,Date,3001,3002,3003,3004,3005,3006,3007,3008,3009,...,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031
0,1/1/17,3624.0337,45.9284,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1/2/17,3540.0400,3785.3424,55.7984,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<font color='green'>**Количество просмотренной рекламы**</font>

In [28]:
total_ads.head(2)

,Date,3011,3016,3017,3005,3004,3007,3006,3001,3014,...,3010,3037,3030,3031,3032,3033,3034,3060,3013,3036
0,1/1/17,0,0,0,0,0,0,0,151902,0,...,0,0,0,0,0,0,0,0,0,0
1,1/2/17,0,0,0,0,0,0,0,157271,0,...,0,0,0,0,0,0,0,0,0,0


<font color='green'>**Доход от рекламы**</font>

In [22]:
ads_revenue = list()
for date in total_ads['Date']:
    ecpm_value=ecpm.loc[pd.to_datetime(ecpm['Date'])==pd.to_datetime(date)]['eCPM']
    row=list()
    row.append(date) # собираем все даты
    for i in list(total_ads.drop('Date', axis=1)):
                                        # кол-во рекламы*ставку по рекламе
            row.append((float(total_ads.\
                              loc[total_ads['Date']==date][i])*float(ecpm_value))/1000)
            
    ads_revenue.append(row) # собираем по всем датам
    
ads_revenue = pd.DataFrame(ads_revenue,columns=list(total_ads))
print('Calculation of Revenue is done')

Calculation of Revenue is done


<font color='green'>**Расчет метрики Life time value**</font>

In [32]:
ltv_all= list()

for cohort in cohorts:
    ltv=0
    row=list()
    row.append(cohort)
    
    if str(cohort) in list(ads_revenue.columns.values) and str(cohort) in list(iap.columns.values):
        
        for date in pd.to_datetime(days):
            ads_revenue_day=float(ads_revenue.loc[pd.to_datetime(ads_revenue['Date'])==date][str(cohort)])
            iap_revenue_day=float(iap.loc[pd.to_datetime(iap['Date'])==date][str(cohort)])
            cohort_installs=float(installs.loc[installs['cohort_number']==cohort]['installs'])
            
            ltv=ltv+((ads_revenue_day+iap_revenue_day)/cohort_installs)
            row.append(ltv)
    else:
        pass
    
    ltv_all.append(row)
#     print(str(cohort)+' is ended')

columns=list()
day=list(days)

for x in list(day):
    columns.append('LTV'+str(day.index(x)+1))
    
columns.insert(0,'cohort')

ltv_all = pd.DataFrame(ltv_all,columns=columns)
print('Calculation of LTV is completed')

Calculation of LTV is completed


In [33]:
# to csv
ltv_all.to_csv('ltv all in all.csv')

print('Uploading to csv is done')

Uploading to csv is done
